In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class Covid19Model0(nn.Module):
  def __init__(self):
    super(Covid19Model0, self).__init__()

    # Convolutional layers
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.dropout1 = nn.Dropout(0.5)  # Dropout after conv1
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.dropout2 = nn.Dropout(0.5)  # Dropout after conv2
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.dropout3 = nn.Dropout(0.5)  # Dropout after conv3

    # Pooling layer
    self.pool = nn.MaxPool2d(2, 2)

    # Fully connected layers
    self.linear1 = nn.Linear(128 * 31 * 31, 256)
    self.dropout4 = nn.Dropout(0.5)  # Dropout after linear1
    self.linear2 = nn.Linear(256, 128)
    self.dropout5 = nn.Dropout(0.5)  # Dropout after linear2
    self.linear3 = nn.Linear(128, 64)
    self.dropout6 = nn.Dropout(0.5)  # Dropout after linear3
    self.linear4 = nn.Linear(64, 3)   # Output layer for 3 classes

  def forward(self, x):
    # Forward pass through the network
    x = F.relu(self.conv1(x))
    x = self.dropout1(x)
    x = self.pool(x)

    x = F.relu(self.conv2(x))
    x = self.dropout2(x)
    x = self.pool(x)

    x = F.relu(self.conv3(x))
    x = self.dropout3(x)
    x = self.pool(x)

    # Flatten the tensor for the fully connected layers
    x = x.view(-1, 128 * 31 * 31)

    x = F.relu(self.linear1(x))
    x = self.dropout4(x)

    x = F.relu(self.linear2(x))
    x = self.dropout5(x)

    x = F.relu(self.linear3(x))
    x = self.dropout6(x)

    x = self.linear4(x)  # Output layer
    return x


In [14]:
model = Covid19Model0()
model.load_state_dict(torch.load('/content/drive/MyDrive/models/m2.pth'))  # Load weights
model.eval()

<ipython-input-14-394a26f8757c>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/models/m2.pth'))  # Load weights


Covid19Model0(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout1): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.5, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout(p=0.5, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=123008, out_features=256, bias=True)
  (dropout4): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (dropout5): Dropout(p=0.5, inplace=False)
  (linear3): Linear(in_features=128, out_features=64, bias=True)
  (dropout6): Dropout(p=0.5, inplace=False)
  (linear4): Linear(in_features=64, out_features=3, bias=True)
)

In [34]:
input_shape = [1, 3, 250, 250]
sample_input = torch.zeros((input_shape))

In [35]:
onnx_model_path = "model.onnx"

# Export the model
torch.onnx.export(
    model,                   # PyTorch model
    sample_input,            # Sample input tensor
    onnx_model_path,         # Path to save the ONNX model
    export_params=True,      # Store the trained parameter weights inside the model file
    opset_version=11,        # ONNX version to export the model to
    do_constant_folding=True # Fold constant nodes for optimization
)

In [20]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


In [41]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# Load the image
# image_path = "/content/drive/MyDrive/Kaggle/Covid19-dataset/test/Viral Pneumonia/0102.jpeg"  # Replace with your image path
# image_path = "/content/drive/MyDrive/Kaggle/Covid19-dataset/test/Normal/0102.jpeg"  # Replace with your image path
image_path = "/content/drive/MyDrive/Kaggle/Covid19-dataset/test/Covid/0102.jpeg"  # Replace with your image path
image = Image.open(image_path)

# Define the transformation pipeline
t = transforms.Compose([
    transforms.Resize((250, 250)),            # Resize image to 256x256
    transforms.RandomHorizontalFlip(p=0.5),   # Randomly flip the image horizontally
    transforms.ToTensor(),                    # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply the transformation
image_tensor = t(image)

# Add a batch dimension (1, C, H, W) if required by the model
image_tensor = image_tensor.unsqueeze(0)
# image_tensor = image_tensor.repeat(8, 1, 1, 1)
print(image_tensor.shape)  # Verify the shape, should be (1, C, H, W)


torch.Size([1, 3, 250, 250])


In [42]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
session = ort.InferenceSession(onnx_model_path)

# Prepare the input data
input_name = session.get_inputs()[0].name
input_data = image_tensor.numpy()  # Convert to NumPy array if necessary

# Run inference
output = session.run(None, {input_name: input_data})

print(output)

output = np.argmax(output, axis=2)

# Display the result
print("Inference output:", output)


[array([[ 2.7040148 ,  0.05735902, -1.1582744 ]], dtype=float32)]
Inference output: [[0]]
